In [1]:
! pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d pythonafroz/solar-panel-images

In [ ]:
from zipfile import ZipFile
dataset = '/content/solar-panel-images.zip'
with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('Done')

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import VGG16,MobileNetV3Small
from tensorflow.keras import layers, models

from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping
from sklearn.metrics import classification_report

In [ ]:
image_data='/content/Faulty_solar_panel'
pd.DataFrame(os.listdir(image_data),columns=['Files_Name'])


In [ ]:
img_height, img_width = 224, 224
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    image_data,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    image_data,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
    layers.RandomBrightness(factor=0.2),
    layers.RandomContrast(0.1),
])

In [ ]:
class_names = train_ds.class_names
print("classes:", class_names)
num_classes = len(class_names)
print("Number of classes:", num_classes)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(500).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # freeze base model

# Add custom classifier on top
inputs = keras.Input(shape=(img_height, img_width, 3))
x = keras.applications.vgg16.preprocess_input(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x= layers.Dense(90)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.summary()

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
epochs = 15
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[
        EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True,verbose=1,min_delta=1e-2)
    ]
)

In [ ]:
# fine tuning
base_model.trainable = True
for layer in base_model.layers[:14]:
    layer.trainable = False
model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(0.0001),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epoch = 30
history = model.fit(train_ds, validation_data=val_ds, epochs=epoch,
    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor="val_loss",
            min_delta=1e-2,
            patience=3,
            verbose=1,
        )
    ]
)

In [ ]:
base_model = keras.applications.VGG16(
    weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3)
)

# Freeze first 15 layers
for layer in base_model.layers[:15]:
    layer.trainable = False

inputs =keras.Input(shape=(img_height, img_width, 3))

x=data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = keras.Model(inputs, outputs)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=5e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

callbacks = [
    keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2, min_lr=1e-7)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=callbacks
)

In [ ]:
model.save("fine_tuned_solar_model2.h5")

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure()
plt.plot(epochs_range, acc, label="Training Accuracy")
plt.plot(epochs_range, val_acc, label="Validation Accuracy")
plt.legend()

plt.figure()
plt.plot(epochs_range, loss, label="Training Loss")
plt.plot(epochs_range, val_loss, label="Validation Loss")
plt.legend()
plt.show()

In [ ]:
loss, accuracy = model.evaluate(val_ds)

In [ ]:
loss, accuracy = model.evaluate(train_ds)

In [ ]:
plt.figure(figsize=(20, 20))
for images, labels in val_ds.take(1):
    for i in range(16):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        predictions = model.predict(tf.expand_dims(images[i], 0))
        score = tf.nn.softmax(predictions[0])
        if(class_names[labels[i]]==class_names[np.argmax(score)]):
            plt.title("Actual: "+class_names[labels[i]])
            plt.ylabel("Predicted: "+class_names[np.argmax(score)],fontdict={'color':'green'})

        else:
            plt.title("Actual: "+class_names[labels[i]])
            plt.ylabel("Predicted: "+class_names[np.argmax(score)],fontdict={'color':'red'})
        plt.gca().axes.yaxis.set_ticklabels([])
        plt.gca().axes.xaxis.set_ticklabels([])

In [ ]:
base_model = MobileNetV3Small(input_shape=(240,240,3),
                              include_top=False,
                              weights='imagenet')

base_model.trainable = False

mobilenet_model=models.Sequential()
mobilenet_model.add(base_model)
mobilenet_model.add(layers.Flatten())
mobilenet_model.add(layers.Dropout(0.3))
mobilenet_model.add(layers.Dense(6,activation=tf.nn.softmax))
mobilenet_model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=['accuracy'])

In [ ]:
lr_reduce_cb = ReduceLROnPlateau(
    factor=0.5,
    patience=2,
)

earlystop_cb = EarlyStopping(
    patience=5,
    restore_best_weights=True,
)

callbacks = [earlystop_cb , lr_reduce_cb]

In [ ]:
mobilenet_history = mobilenet_model.fit(train_dataset, epochs=10, validation_data=val_dataset ,callbacks=callbacks)